In [146]:
import yfinance as yf
import pandas as pd
from matplotlib import pyplot as plt
import plotly.express as px
from datetime import date

from Functions import date_years_ago
from Stock import Stock

from sklearn.model_selection import train_test_split
import statsmodels.tsa.api as tsa
from statsmodels.graphics import tsaplots
import torch
import torch.nn as nn


In [53]:
ticker = yf.Ticker("AAPL")
data = yf.download("AAPL")

if ticker.info['regularMarketPrice']:
    print(ticker.info['regularMarketPrice'])
else:
    print("Error")


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


275.25


In [67]:
dividends = ticker.dividends
info = ticker.info

print(dividends)
print(info.get("dividendRate"), info.get("dividendYield"))

Date
1987-05-11 00:00:00-04:00    0.000536
1987-08-10 00:00:00-04:00    0.000536
1987-11-17 00:00:00-05:00    0.000714
1988-02-12 00:00:00-05:00    0.000714
1988-05-16 00:00:00-04:00    0.000714
                               ...   
2024-11-08 00:00:00-05:00    0.250000
2025-02-10 00:00:00-05:00    0.250000
2025-05-12 00:00:00-04:00    0.260000
2025-08-11 00:00:00-04:00    0.260000
2025-11-10 00:00:00-05:00    0.260000
Name: Dividends, Length: 89, dtype: float64
1.04 0.38


In [85]:
stock = Stock('GOOG')

dividends_df = stock.get_dividends()
dividends_df

Date
2024-06-10 00:00:00-04:00    0.20
2024-09-09 00:00:00-04:00    0.20
2024-12-09 00:00:00-05:00    0.20
2025-03-10 00:00:00-04:00    0.20
2025-06-09 00:00:00-04:00    0.21
2025-09-08 00:00:00-04:00    0.21
Name: Dividends, dtype: float64

In [164]:
# test function
def plot_dividends(div_df: pd.DataFrame):
    """ forecast future dividends (and maybe when dividends will increase based on revenue/income/stock price) """
    fig = px.line(data_frame=div_df, x='Date', y='Dividends', color='Company', markers=True)
    fig.show()

# div_df = stock.get_dividends().reset_index()
# plot_dividends(div_df)

In [159]:
# test function

from Database.database import engine
from Database.models import Holding
from sqlalchemy import select

with engine.connect() as conn:
    holdings_df = pd.read_sql(select(Holding), conn)

dividends_df = pd.DataFrame(columns=['Date','Dividends', 'Company'])

for ticker in holdings_df['ticker']:
    stock = Stock(ticker)

    divs_df = stock.get_dividends().reset_index()
    divs_df['Company'] = ticker

    dividends_df = pd.concat([dividends_df, divs_df])

# dividends_df

2025-11-12 19:43:28,089 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-11-12 19:43:28,091 INFO sqlalchemy.engine.Engine SELECT holdings.id, holdings.ticker, holdings."holdingType", holdings."holdingSize", holdings."dateAdded", holdings."dateEdited" 
FROM holdings
2025-11-12 19:43:28,093 INFO sqlalchemy.engine.Engine [cached since 1.178e+05s ago] ()
2025-11-12 19:43:28,097 INFO sqlalchemy.engine.Engine ROLLBACK


C:\Users\Anay-\AppData\Local\Temp\ipykernel_32128\3464183309.py:18: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



,Date,Dividends,Company
0,1987-05-11 00:00:00-04:00,0.000536,AAPL
1,1987-08-10 00:00:00-04:00,0.000536,AAPL
2,1987-11-17 00:00:00-05:00,0.000714,AAPL
3,1988-02-12 00:00:00-05:00,0.000714,AAPL
4,1988-05-16 00:00:00-04:00,0.000714,AAPL
...,...,...,...
84,2024-11-08 00:00:00-05:00,0.250000,AAPL
85,2025-02-10 00:00:00-05:00,0.250000,AAPL
86,2025-05-12 00:00:00-04:00,0.260000,AAPL
87,2025-08-11 00:00:00-04:00,0.260000,AAPL


In [165]:
plot_dividends(dividends_df)

In [10]:
holdings_df = pd.DataFrame([], columns=['Ticker', 'Holding Type', 'Holding Size'])

holdings_df.to_csv('holdings_df.csv')


In [12]:
def message_handler(message):
    print("Received message: ", message)


In [13]:
def market_info(market_name):
    market = yf.Market(market_name)
    status = market.status
    summary = market.summary

    print(status)
    print(summary)

In [14]:
def options(ticker_name):
    ticker = yf.Ticker(ticker_name)

    return ticker.option_chain(ticker.options[0]).calls


In [15]:
def general_info(ticker_name):
    ticker_name = "AAPL"

    ticker = yf.Ticker(ticker_name)

    print(ticker.info)



In [ ]:
ticker = yf.Ticker("AAPL")
tickers = yf.Tickers(' aapl msft goog')

In [ ]:
analyst_price_targets = ticker.analyst_price_targets
calendar = ticker.calendar


In [131]:
data = yf.download("AAPL", period="10y", interval="1d")
data

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2015-11-12,26.091269,26.339284,26.075486,26.213022,130102400
2015-11-13,25.329182,26.057447,25.313399,25.974023,183249600
2015-11-16,25.744045,25.757573,25.027054,25.112731,152426800
2015-11-17,25.633566,25.940204,25.550142,25.910892,110467600
2015-11-18,26.445253,26.490346,26.041664,26.100286,186698800
...,...,...,...,...,...
2025-11-05,269.878387,271.436874,266.671474,268.349840,43683100
2025-11-06,269.508728,273.135217,267.630574,267.630574,51204000


In [ ]:
plt.plot(appl_df['Close'])
plt.show()

tsaplots.acf(appl_df['Close']) # also try appl_df

# tsaplots.acf(appl_df)

plt.show()